In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "MIG-05a30703-b842-5a2b-9c11-02cc84a8a7e4"
# Display the value of cuda_device
print("CUDA device:", os.environ["CUDA_VISIBLE_DEVICES"] )

CUDA device: MIG-05a30703-b842-5a2b-9c11-02cc84a8a7e4


In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
from datasets import load_dataset, Dataset
from transformers import Trainer, TrainingArguments
import numpy as np

# 1. Setup & Imports

# Check if CUDA is available and if not, use CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)


# 2. Dataset Preparation

# Load CIFAR-10 dataset
dataset = load_dataset("cifar10")


# Image Transformation
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)) #normalize for ResNet
])


def transform_images(examples):
    images = [transform(image.convert("RGB")) for image in examples['img']]
    return {"pixel_values": images, "labels": examples['label']}


# Apply transformation and convert to HF Dataset
train_dataset = dataset["train"].map(transform_images, batched=True, remove_columns=["img"])
test_dataset = dataset["test"].map(transform_images, batched=True, remove_columns=["img"])

# Set output format to torch to not mess up Trainer
train_dataset.set_format("torch", columns=["pixel_values","labels"])
test_dataset.set_format("torch", columns=["pixel_values","labels"])

print(train_dataset)
print(test_dataset)
print("Sample dataset output:", train_dataset[0]) # Double check this


# 3. Model Definition

class BasicBlock(nn.Module):
    expansion = 1
    def __init__(self, in_channels, out_channels, stride=1):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_channels != self.expansion*out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, self.expansion*out_channels, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*out_channels)
            )

    def forward(self, x):
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        out += self.shortcut(identity)
        out = self.relu(out)
        return out


class ResNet(nn.Module):
    def __init__(self, block, layers, num_classes=10):
        super().__init__()
        self.in_channels = 64
        self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

        self.layer1 = self._make_layer(block, 64, layers[0], stride=1)
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2)
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2)
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2)

        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512 * block.expansion, num_classes)


    def _make_layer(self, block, out_channels, num_blocks, stride):
        strides = [stride] + [1] * (num_blocks - 1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_channels, out_channels, stride))
            self.in_channels = out_channels * block.expansion
        return nn.Sequential(*layers)

    def forward(self, inputs):  # Expecting a dictionary
        x = inputs["pixel_values"]  # Retrieve "pixel_values" from input dictionary
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.maxpool(out)

        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)

        out = self.avgpool(out)
        out = torch.flatten(out, 1)
        out = self.fc(out)
        return out


def resnet18(num_classes=10):
    return ResNet(BasicBlock, [2, 2, 2, 2], num_classes=num_classes)


model = resnet18()
model.to(device)

# 4. Training Configuration

training_args = TrainingArguments(
    output_dir="./resnet_cifar10",
    per_device_train_batch_size=64,
    per_device_eval_batch_size=64,
    num_train_epochs=10,
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_dir="./logs",
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    learning_rate=1e-3,
    weight_decay=1e-4,
    remove_unused_columns = False # Removed this argument
)

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    accuracy = np.mean(predictions == labels)
    return {"accuracy": accuracy}


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics
)

# 5. Training Loop

trainer.train()

Using device: cuda
Dataset({
    features: ['label', 'pixel_values', 'labels'],
    num_rows: 50000
})
Dataset({
    features: ['label', 'pixel_values', 'labels'],
    num_rows: 10000
})
Sample dataset output: {'pixel_values': tensor([[[ 0.3961,  0.3961,  0.3961,  ...,  0.3333,  0.3176,  0.2941],
         [ 0.4118,  0.4039,  0.4118,  ...,  0.3569,  0.3412,  0.3176],
         [ 0.3882,  0.3882,  0.3961,  ...,  0.3412,  0.3255,  0.3098],
         ...,
         [-0.1216, -0.1137, -0.1059,  ..., -0.2157, -0.2314, -0.2078],
         [-0.1216, -0.1216, -0.1137,  ..., -0.2000, -0.2000, -0.2000],
         [-0.1922, -0.2157, -0.1922,  ..., -0.2784, -0.2706, -0.2863]],

        [[ 0.3804,  0.3804,  0.3804,  ...,  0.3176,  0.3020,  0.2784],
         [ 0.3961,  0.3882,  0.3961,  ...,  0.3412,  0.3255,  0.3020],
         [ 0.3725,  0.3725,  0.3804,  ...,  0.3255,  0.3098,  0.2941],
         ...,
         [-0.1608, -0.1451, -0.1373,  ..., -0.2392, -0.2627, -0.2549],
         [-0.2000, -0.1922, -0.19

Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
CODECARBON : No CPU tracking mode found. Falling back on CPU constant mode.
CODECARBON : Failed to match CPU TDP constant. Falling back on a global constant.


TypeError: ResNet.forward() got an unexpected keyword argument 'pixel_values'

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
from datasets import load_dataset, Dataset
from transformers import Trainer, TrainingArguments
import numpy as np

# 1. Setup & Imports

# Check if CUDA is available and if not, use CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)


# 2. Dataset Preparation

# Load CIFAR-10 dataset
dataset = load_dataset("cifar10")

Using device: cuda


In [3]:
dataset

DatasetDict({
    train: Dataset({
        features: ['img', 'label'],
        num_rows: 50000
    })
    test: Dataset({
        features: ['img', 'label'],
        num_rows: 10000
    })
})

In [4]:
dataset['train'][0]

{'img': <PIL.PngImagePlugin.PngImageFile image mode=RGB size=32x32>,
 'label': 0}

In [5]:

# Image Transformation
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)) #normalize for ResNet
])


def transform_images(examples):
    images = [transform(image.convert("RGB")) for image in examples['img']]
    return {"pixel_values": images, "labels": examples['label']}


# Apply transformation and convert to HF Dataset
train_dataset = dataset["train"].map(transform_images, batched=True, remove_columns=["img"])
test_dataset = dataset["test"].map(transform_images, batched=True, remove_columns=["img"])


In [6]:

# Set output format to torch to not mess up Trainer
train_dataset.set_format("torch", columns=["pixel_values","labels"])
test_dataset.set_format("torch", columns=["pixel_values","labels"])

print(train_dataset)
print(test_dataset)

Dataset({
    features: ['label', 'pixel_values', 'labels'],
    num_rows: 50000
})
Dataset({
    features: ['label', 'pixel_values', 'labels'],
    num_rows: 10000
})


In [8]:

# 3. Model Definition

class BasicBlock(nn.Module):
    expansion = 1
    def __init__(self, in_channels, out_channels, stride=1):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_channels != self.expansion*out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, self.expansion*out_channels, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*out_channels)
            )

    def forward(self, x):
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        out += self.shortcut(identity)
        out = self.relu(out)
        return out


class ResNet(nn.Module):
    def __init__(self, block, layers, num_classes=10):
        super().__init__()
        self.in_channels = 64
        self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

        self.layer1 = self._make_layer(block, 64, layers[0], stride=1)
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2)
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2)
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2)

        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512 * block.expansion, num_classes)


    def _make_layer(self, block, out_channels, num_blocks, stride):
        strides = [stride] + [1] * (num_blocks - 1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_channels, out_channels, stride))
            self.in_channels = out_channels * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x):
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.maxpool(out)

        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)

        out = self.avgpool(out)
        out = torch.flatten(out, 1)
        out = self.fc(out)
        return out


def resnet18(num_classes=10):
    return ResNet(BasicBlock, [2, 2, 2, 2], num_classes=num_classes)


model = resnet18()
model.to(device)

# 4. Training Configuration


training_args = TrainingArguments(
     output_dir="./resnet_cifar10",
     per_device_train_batch_size=64,
     per_device_eval_batch_size=64,
     num_train_epochs=10,
     eval_strategy="epoch",  # Changed this
     save_strategy="epoch",
     logging_dir="./logs",
     logging_steps=10,
     remove_unused_columns=False,
     load_best_model_at_end=True,
     metric_for_best_model="accuracy",
     learning_rate=1e-3,
     weight_decay=1e-4
 )

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    accuracy = np.mean(predictions == labels)
    return {"accuracy": accuracy}


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics
)

Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
CODECARBON : No CPU tracking mode found. Falling back on CPU constant mode.
CODECARBON : Failed to match CPU TDP constant. Falling back on a global constant.


In [9]:

# 5. Training Loop

trainer.train()

TypeError: ResNet.forward() got an unexpected keyword argument 'pixel_values'